In [1]:
import os
import numpy as np
import pandas as pd
import SimpleITK as sitk
from radiomics import featureextractor
import itertools
import re


In [2]:
# Py-radiomics extractor that extract only shape features.
extractor = featureextractor.RadiomicsFeatureExtractor()
extractor.settings['additionalInfo'] = False
extractor.disableAllImageTypes()
extractor.enableImageTypeByName('Original')
extractor.disableAllFeatures()
extractor.enableFeatureClassByName('shape')

In [3]:
print("Number of shape feature:", len(extractor.enabledFeatures['shape']))  # 14

Number of shape feature: 0


In [4]:
BASE       = os.getcwd()
TRAIN_DIR  = os.path.join(BASE, "Dataset", "Train")
TEST_DIR   = os.path.join(BASE, "Dataset", "SegTest")
OUT_DIR    = os.path.join(BASE, "data")
os.makedirs(OUT_DIR, exist_ok=True)

In [5]:
def extract_per_patient(folder, sid):
    rec = {"Id": sid}
    for ph in ["ED", "ES"]:
        img = sitk.ReadImage(os.path.join(folder, f"{sid}_{ph}.nii"))
        msk = sitk.ReadImage(os.path.join(folder, f"{sid}_{ph}_seg.nii"))
        for lbl, name in [(1,"RV"),(3,"LV"),(2,"MY")]:
            try:
                feats = extractor.execute(img, msk, label=lbl)
                for k,v in feats.items():
                    rec[f"{ph}_{name}_{k.replace('original_','')}"] = v
            except ValueError:
                # label not present → fill NaN for all 14 shape metrics
                for k in extractor.enabledFeatures['shape']:
                    rec[f"{ph}_{name}_{k}"] = np.nan
    return rec

In [6]:
def all_patient(src_dir):
    rows = [extract_per_patient(os.path.join(src_dir,s), s)
            for s in os.listdir(src_dir) if os.path.isdir(os.path.join(src_dir,s))]
    df = pd.DataFrame(rows)
    return df

In [7]:
def add_ratios(df,Train=True):
    
    # --- 1 · isolate volume columns ---------------------------------------------
    vol_cols = [c for c in df.columns if re.search(r'MeshVolume$', c)]
    # e.g. ['ED_LV_shape_Volume', 'ES_LV_shape_Volume', 'ED_RV_shape_Volume', …]
    print(vol_cols)
    # --- 2 · pair-wise ratios with itertools.combinations -----------------------
    ratio_frames = {}          # will hold Series objects keyed by new column name

    for a, b in itertools.combinations(vol_cols, 2):
        name = f"{a}_over_{b}"              # ED_LV_shape_Volume_over_ES_LV_shape_Volume
        ratio = np.divide(df[a], df[b])
        ratio_frames[name] = ratio

    # merge them into the table
    df = pd.concat([df, pd.DataFrame(ratio_frames)], axis=1)

    return df
 


In [8]:

def compute_body_surface_area(height,weight): 
    "Return the body surface area from height and weight. (Formula of Du Bois)"
    
    return 0.007184 * (height**0.725 )* (weight**0.425)

def add_body_surface_area_feature(df : pd.DataFrame ,name_column_height = "Height",name_column_weight = "Weight"):
    # Description :
    # Add for each row the BSA associated.
    
    if (name_column_height and name_column_weight in df.columns) and ("body_surface" not in df.columns)  :
        df["body_surface"] = compute_body_surface_area(df[name_column_height],df[name_column_weight])
        #print("Body surface area feature added")
    else : 
        print("provide a dataframe with a height and weight feature")
       


In [13]:
def add_meta_data(df,Train= True):  
    if Train : 
        metaData = pd.read_csv(os.path.join(BASE,"Dataset","metaDataTrain.csv", usecols=["Id", "Height", "Weight"]))
    else :
        metaData = pd.read_csv(os.path.join(BASE,"Dataset","metaDataTest.csv", usecols=["Id", "Height", "Weight"]))

    df["Id"]        = df["Id"].astype(str)
    metaData["Id"]  = metaData["Id"].astype(str)

    # ── jointure -----------------------------------------------------------
    df = df.merge(metaData, on="Id", how="left")
    
    add_body_surface_area_feature(df)           
    df.drop(columns=["Height", "Weight"], errors="ignore", inplace=True)
    
    return df

In [14]:

df_train = all_patient(TRAIN_DIR)
df_test = all_patient(TEST_DIR)


NiftiImageIO (0x12b094f40): /Users/rplanchon/Documents/telecom/IMA/S2/IMA205/Challenge/CardiacPathoPrediction/Dataset/Train/024/024_ED.nii has unexpected scales in sform

NiftiImageIO (0x12b094f40): /Users/rplanchon/Documents/telecom/IMA/S2/IMA205/Challenge/CardiacPathoPrediction/Dataset/Train/024/024_ED.nii has unexpected scales in sform

NiftiImageIO (0x1280ec930): /Users/rplanchon/Documents/telecom/IMA/S2/IMA205/Challenge/CardiacPathoPrediction/Dataset/Train/024/024_ED_seg.nii has unexpected scales in sform

NiftiImageIO (0x1280ec930): /Users/rplanchon/Documents/telecom/IMA/S2/IMA205/Challenge/CardiacPathoPrediction/Dataset/Train/024/024_ED_seg.nii has unexpected scales in sform

NiftiImageIO (0x12b09fa10): /Users/rplanchon/Documents/telecom/IMA/S2/IMA205/Challenge/CardiacPathoPrediction/Dataset/Train/024/024_ES.nii has unexpected scales in sform

NiftiImageIO (0x12b09fa10): /Users/rplanchon/Documents/telecom/IMA/S2/IMA205/Challenge/CardiacPathoPrediction/Dataset/Train/024/024_ES.ni

In [15]:

df_train = add_ratios(df_train,Train=True)
df_test = add_ratios(df_test,Train=False)


['ED_RV_shape_MeshVolume', 'ED_LV_shape_MeshVolume', 'ED_MY_shape_MeshVolume', 'ES_RV_shape_MeshVolume', 'ES_LV_shape_MeshVolume', 'ES_MY_shape_MeshVolume']
['ED_RV_shape_MeshVolume', 'ED_LV_shape_MeshVolume', 'ED_MY_shape_MeshVolume', 'ES_RV_shape_MeshVolume', 'ES_LV_shape_MeshVolume', 'ES_MY_shape_MeshVolume']


In [16]:

df_train = add_meta_data(df_train,Train=True)
df_test = add_meta_data(df_test,Train=False)


In [17]:

df_train.to_csv(os.path.join(OUT_DIR,"TrainningDataset.csv"), index=False)
df_test.to_csv(os.path.join(OUT_DIR,"TestingDataset.csv"), index=False)